# Лекция №10: Основы и продвинутые методы Pandas

### Цели лекции

1.  **Освоить основы Pandas:** изучить структуры `Series` и `DataFrame`, научиться загружать данные из CSV и проводить первичный анализ (`.head()`, `.info()`, `.describe()`).
2.  **Научиться манипулировать данными:** освоить выборку, создание, удаление столбцов и строк (`.loc`, `.iloc`), а также фильтрацию по условиям.
3.  **Научиться работать с отсутствующими данными:** освоить методы их обнаружения (`.isNone()`), удаления (`.dropna()`) и заполнения (`.fillna()`).
4.  **Познакомиться с продвинутыми техниками:** получить представление об агрегации данных с помощью `.groupby()` и применении пользовательских функций через `.apply()`.

## 1. Что такое Pandas и зачем он нужен?

**Pandas** — это высокоуровневая библиотека Python, созданная для анализа и манипуляции данными. Она построена поверх NumPy и является стандартом де-факто в Data Science.

**Почему Pandas так важен?**
- **Скорость и эффективность:** Хотя Pandas написан на Python, его критически важные части реализованы на C, что обеспечивает высокую производительность.
- **Удобство:** Pandas предоставляет мощные и простые в использовании структуры данных для работы с таблицами (похожими на Excel или SQL-таблицы).
- **Гибкость:** Позволяет читать и записывать данные из множества форматов: CSV, Excel, SQL, JSON и других.

> **Простыми словами:** если NumPy — это "арифметика" для матриц, то Pandas — это "Excel на стероидах" прямо в коде Python, созданный специально для подготовки данных к машинному обучению.

### Импорт библиотек

По общепринятому соглашению, Pandas импортируется под псевдонимом `pd`.

In [ ]:
import pandas as pd
import numpy as np

## 2. Структуры данных: Series и DataFrame

В Pandas есть две основные структуры данных, которые являются его "алфавитом".

### 2.1. Series

**`Series`** — это одномерный массив с метками (индексом). Его можно представить как один столбец в таблице.

Ключевое отличие от массива NumPy — наличие **именованного индекса**. Это позволяет обращаться к элементам не только по числовой позиции, но и по метке, как в словаре Python.

In [ ]:
my_data = [1776, 1867, 1821]
my_index = ['USA', 'Canada', 'Mexico']

my_series = pd.Series(data=my_data, index=my_index)
print(my_series)

### 2.2. DataFrame

**`DataFrame`** — это основная структура данных в Pandas. Это двумерная таблица, состоящая из строк и столбцов. 

> **Простая аналогия:** `DataFrame` — это таблица Excel или SQL. Каждый столбец в этом `DataFrame` является объектом `Series`, и все эти столбцы-`Series` имеют общий индекс.

In [ ]:
# Создаем DataFrame из случайных чисел NumPy
np.random.seed(101) # для воспроизводимости результатов
data = np.random.randn(3,4) # 3 строки, 4 столбца
index = ['A', 'B', 'C']
columns = ['W', 'X', 'Y', 'Z']

df = pd.DataFrame(data=data, index=index, columns=columns)
df

## 3. Загрузка данных: `pd.read_csv()`

Чаще всего вы не будете создавать DataFrame вручную. Вместо этого вы будете загружать данные из внешнего источника. Самый распространенный формат — это CSV (Comma-Separated Values, значения, разделенные запятыми).

Для этого используется функция `pd.read_csv()`.

In [ ]:
# Загружаем датасет с информацией о чаевых в ресторане
# Файл 'tips.csv' должен находиться в той же папке, что и этот блокнот
tips_df = pd.read_csv('tips.csv')

## 4. Первичный осмотр данных

После загрузки данных первое, что нужно сделать, — это провести "осмотр пациента". Для этого есть несколько критически важных методов.

### `.head()` — посмотреть на первые строки
Этот метод позволяет увидеть "шапку" таблицы и получить первое представление о том, как выглядят данные. Последние строки позволяет смотреть метод `.tail()`

In [ ]:
# По умолчанию .head() показывает 5 первых строк
tips_df.head()

### `.info()` — структура и типы данных

Этот метод дает краткую сводку о DataFrame: количество строк, количество и названия столбцов, количество непустых значений (`non-None`) и типы данных в каждом столбце. Это **первый и главный способ обнаружить пропуски в данных**.

In [ ]:
tips_df.info()

### `.describe()` — основные статистические показатели

Этот метод рассчитывает описательные статистики **только для числовых столбцов**: количество, среднее, стандартное отклонение, минимум, максимум и перцентили.

In [ ]:
tips_df.describe()

## 5. Работа с отсутствующими данными (Missing Data)

В реальных наборах данных практически всегда встречаются пропуски. Они могут возникать из-за ошибок сбора данных, неполных записей или просто потому, что какая-то информация была недоступна. Большинство алгоритмов машинного обучения не умеют работать с пропусками, поэтому их необходимо обработать.

В Pandas пропущенные значения обычно представляются как `NaN` (Not a Number).

**Основные стратегии работы с пропусками:**
1.  **Удалить:** Можно удалить либо строки, либо столбцы с пропусками.
2.  **Заполнить:** Заменить пропуски некоторым значением (например, нулем, средним, медианой или модой).

In [ ]:
data = {'A': [1, 2, np.nan, 4],
        'B': [5, np.nan, np.nan, 8],
        'C': [9, 10, 11, 12]}
df_missing = pd.DataFrame(data)
df_missing

### 5.1. Обнаружение и удаление пропусков

- `.isNone()`: возвращает DataFrame с `True` на месте пропусков.
- `.dropna()`: удаляет строки (по умолчанию) или столбцы (`axis=1`), содержащие `NaN`.

In [ ]:
# Посчитать количество пропусков в каждом столбце (True = 1, False = 0)
df_missing.isNone().sum()

In [ ]:
# Удаление любой строки, содержащей хотя бы один пропуск
df_missing.dropna()

In [ ]:
# Удаление любого столбца, содержащего хотя бы один пропуск
df_missing.dropna(axis=1)

### 5.2. Заполнение пропусков `.fillna()`

Более гибкий подход — заполнение пропусков.

In [ ]:
# Заполнение всех пропусков одним значением, например, нулем
df_missing.fillna(0)

Часто пропуски заполняют средним значением по столбцу. Это позволяет сохранить общую статистику данных.

In [ ]:
# 1. Вычисляем среднее для столбца 'A'
mean_A = df_missing['A'].mean()
print(f"Среднее для столбца A: {mean_A:.2f}")

# 2. Заполняем пропуски в 'A' этим средним
# inplace=True изменяет исходный DataFrame, а не возвращает копию
df_missing['A'].fillna(mean_A, inplace=True) 
df_missing

## 6. Работа со столбцами и строками

### 6.1. Выбор, создание и удаление столбцов

In [ ]:
# Выбор одного столбца (результат - Series)
print("--- Один столбец ---")
print(tips_df['total_bill'].head())

# Выбор нескольких столбцов (результат - DataFrame)
print("\n--- Несколько столбцов ---")
print(tips_df[['total_bill', 'sex']].head())

In [ ]:
# Создание нового столбца на основе существующих
tips_df['tip_percentage'] = 100 * tips_df['tip'] / tips_df['total_bill']
tips_df.head()

In [ ]:
# Удаление столбца. axis=1 - обязательно!
# Чтобы удалить столбец из исходного DataFrame, нужно добавить inplace=True
tips_df.drop('tip_percentage', axis=1, inplace=True)
tips_df.head()

### 6.2. Выборка строк: `.loc` и `.iloc`
- `.loc[]` — **label-based** indexing (индексация по меткам/именам).
- `.iloc[]` — **integer-location** based indexing (индексация по числовой позиции).

In [ ]:
# Выборка строки по ее порядковому номеру (индексация с 0)
tips_df.iloc[3]

In [ ]:
# Выборка нескольких строк и определенных столбцов
# Синтаксис: .loc[строки, столбцы]
tips_df.loc[0:3, ['total_bill', 'tip']]

## 7. Фильтрация данных по условию

Это, возможно, самый важный навык в Pandas. Он позволяет выбирать подмножество данных, которое удовлетворяет определенным условиям.

Для комбинирования условий используются операторы `&` (И), `|` (ИЛИ). **Каждое условие должно быть в круглых скобках.**

In [ ]:
# Найдем все заказы, сделанные мужчинами ('sex' == 'Male') в воскресенье ('day' == 'Sun')
tips_df[(tips_df['sex'] == 'Male') & (tips_df['day'] == 'Sun')].head()

## 8. Агрегация и продвинутые манипуляции

### 8.1. Агрегация данных: Group By

Операция `Group By` (группировка) позволяет реализовать парадигму **Split-Apply-Combine** (Разделить-Применить-Объединить):

1.  **Split:** Данные разделяются на группы по категориям.
2.  **Apply:** К каждой группе применяется функция (например, `sum`, `mean`).
3.  **Combine:** Результаты объединяются в новый DataFrame.

In [ ]:
df_cars = pd.read_csv('mpg.csv')
# Найдем среднее значение расхода топлива ('mpg') для каждого количества цилиндров ('cylinders')
avg_mpg_by_cyl = df_cars.groupby('cylinders')['mpg'].mean()
avg_mpg_by_cyl

In [ ]:
# Можно получить и полную статистику по группам
df_cars.groupby('origin')['horsepower'].describe()

### 8.2. Метод `.apply()`

Метод `.apply()` — это ваш "швейцарский нож" для тех случаев, когда встроенных функций Pandas не хватает. Он позволяет применить любую вашу функцию к каждому элементу столбца.

In [ ]:
# Создадим функцию для классификации веса автомобиля
def classify_weight(weight):
    if weight < 2000:
        return 'Легкий'
    elif 2000 <= weight < 3500:
        return 'Средний'
    else:
        return 'Тяжелый'

# Применим ее к столбцу 'weight' и создадим новый столбец
df_cars['weight_class'] = df_cars['weight'].apply(classify_weight)
df_cars[['name', 'weight', 'weight_class']].head()

## Итог

Pandas — это основной инструмент для работы с табличными данными. Навыки загрузки, осмотра, фильтрации и обработки пропусков являются фундаментальными для любого специалиста по данным. Продвинутые методы, такие как `groupby` и `apply`, открывают возможности для сложного и гибкого анализа.